In [1]:
from pyspark.sql import SparkSession
def get_spark():
    spark = SparkSession.builder.appName('SparkDelta') \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.jars.packages",
                "org.apache.hadoop:hadoop-aws:3.2.2,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.180") \
        .getOrCreate()
# This is mandate config on spark session to use AWS S3
    spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
    spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")
# spark.sparkContext.setLogLevel("DEBUG")
    return spark
spark = get_spark()
print("Spark object created")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1666855970919_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark object created

In [2]:
# File location and type
file_location = "s3a://youtube-dataset-kshitij/input/category/*.json"
file_type = "json"

# options
infer_schema = "false"
first_row_is_header = "false"

category_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("multiline","true") \
  .load(file_location)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
category_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                etag|               items|                kind|
+--------------------+--------------------+--------------------+
|"m2yskBQFythfE4ir...|[{"m2yskBQFythfE4...|youtube#videoCate...|
|"XI7nbFXulYBIpL0a...|[{"XI7nbFXulYBIpL...|youtube#videoCate...|
|"ld9biNPKjAjgjV7E...|[{"ld9biNPKjAjgjV...|youtube#videoCate...|
|"ld9biNPKjAjgjV7E...|[{"ld9biNPKjAjgjV...|youtube#videoCate...|
+--------------------+--------------------+--------------------+

In [4]:
category_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)

In [5]:
from pyspark.sql import functions as f
all_category_df = category_df.withColumn("items_exploded",f.explode("items")) \
    .drop('items') \
    .select("items_exploded.id","items_exploded.kind","items_exploded.snippet.title","items_exploded.snippet.channelId")
all_category_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+----------------+--------------------+
| id|                kind|           title|           channelId|
+---+--------------------+----------------+--------------------+
|  1|youtube#videoCate...|Film & Animation|UCBR8-60-B28hp2Bm...|
|  2|youtube#videoCate...|Autos & Vehicles|UCBR8-60-B28hp2Bm...|
| 10|youtube#videoCate...|           Music|UCBR8-60-B28hp2Bm...|
| 15|youtube#videoCate...|  Pets & Animals|UCBR8-60-B28hp2Bm...|
| 17|youtube#videoCate...|          Sports|UCBR8-60-B28hp2Bm...|
+---+--------------------+----------------+--------------------+
only showing top 5 rows

In [6]:
all_category_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

125

In [7]:
distinct_all_category_df = all_category_df.distinct()
distinct_all_category_df.show()
distinct_all_category_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------------------+--------------------+
| id|                kind|               title|           channelId|
+---+--------------------+--------------------+--------------------+
| 31|youtube#videoCate...|     Anime/Animation|UCBR8-60-B28hp2Bm...|
|  2|youtube#videoCate...|    Autos & Vehicles|UCBR8-60-B28hp2Bm...|
| 44|youtube#videoCate...|            Trailers|UCBR8-60-B28hp2Bm...|
| 39|youtube#videoCate...|              Horror|UCBR8-60-B28hp2Bm...|
| 19|youtube#videoCate...|     Travel & Events|UCBR8-60-B28hp2Bm...|
| 30|youtube#videoCate...|              Movies|UCBR8-60-B28hp2Bm...|
| 36|youtube#videoCate...|               Drama|UCBR8-60-B28hp2Bm...|
| 18|youtube#videoCate...|        Short Movies|UCBR8-60-B28hp2Bm...|
| 27|youtube#videoCate...|           Education|UCBR8-60-B28hp2Bm...|
| 21|youtube#videoCate...|       Videoblogging|UCBR8-60-B28hp2Bm...|
| 33|youtube#videoCate...|            Classics|UCBR8-60-B28hp2Bm...|
| 24|youtube#videoCate...|       E

In [8]:
final_df = distinct_all_category_df.select("id","title")
final_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+
| id|               title|
+---+--------------------+
| 31|     Anime/Animation|
|  2|    Autos & Vehicles|
| 44|            Trailers|
| 39|              Horror|
| 19|     Travel & Events|
| 30|              Movies|
| 36|               Drama|
| 18|        Short Movies|
| 27|           Education|
| 21|       Videoblogging|
| 33|            Classics|
| 24|       Entertainment|
| 28|Science & Technology|
| 32|    Action/Adventure|
| 22|      People & Blogs|
| 38|             Foreign|
| 10|               Music|
| 42|              Shorts|
| 17|              Sports|
|  1|    Film & Animation|
+---+--------------------+
only showing top 20 rows

In [9]:
#Casting the column 'id' to int and printing schema
final_df = final_df.select(final_df.id.cast("int"),"title")
final_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)

In [10]:
final_df.write.parquet("s3a://youtube-dataset-kshitij/output/category")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…